In [9]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import seaborn as sns

basket = pd.read_csv('basket-sample.csv')
basket[basket['ビール'] > 0]

,パン,牛乳,ハム,果物,オムツ,ビール,ソーセージ,弁当,タバコ,ジュース
1,1,0,1,0,1,1,0,0,0,0
2,0,0,0,0,0,1,1,0,0,0
3,0,0,0,0,1,1,0,1,1,0
4,0,0,0,1,0,1,0,1,0,1


In [10]:
# ビールとオムツについてブール値を求める
beer = basket['ビール'] > 0
omutsu = basket['オムツ'] > 0
# たとえばビールの支持度（抜き出した数を全体の数で割る）
supp_beer = len(basket[beer]) / len(basket)
# & を使うと，論理積（ビールかつオムツ）を求められる
both = beer & omutsu
# ビール→オムツの確信度
conf_beer_omutsu =len(basket[both]) / len(basket[beer])

Apriori 算法是一种用于挖掘关联规则的经典算法，常用于发现大规模数据集中的频繁项集。关联规则是描述数据中项之间关系的规则，通常用于发现数据集中的潜在模式。

Apriori 算法基于下面的先验原理（Apriori 原理）：

Apriori 原理： 如果一个项集是频繁的，那么它的所有子集也是频繁的。反之，如果一个项集是非频繁的，那么它的所有超集也是非频繁的。

扫描数据集： 统计每个项的支持度，得到频繁 1-项集。

迭代生成频繁 k-项集： 基于频繁 (k-1)-项集，通过连接和剪枝生成频繁 k-项集，直至无法生成更多频繁项集。

生成关联规则： 基于频繁项集，计算关联规则的置信度，选择满足最小支持度和最小置信度要求的规则。


In [18]:
%run apriori.py
groceries = pd.read_csv('groceries.csv', index_col=0)
groceries_rules =apriori(groceries, min_supp=0.05,min_conf=0.1)
groceries_rules.sort_values('Support', ascending=False)
groceries_rules['LHS'].str.contains("milk")

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9      True
10    False
11     True
12    False
13     True
Name: LHS, dtype: bool

最小支持度 (min_supp):

定义： 支持度是一个项集（一组项）在数据集中出现的频率的度量。
目的： min_supp 设定了一个最小支持度阈值，该阈值用于确定一个项集是否被认为是频繁的。支持度高于此阈值的项集被认为是频繁项集。
影响： min_supp 较高的值将导致发现较少的频繁项集，因为它需要更高的支持度。较低的值将允许更多的项集被视为频繁。
最小置信度 (min_conf):

定义： 置信度衡量关联规则的可靠性。对于规则 A → B，置信度是在 A 的条件下发生 B 的概率。
目的： min_conf 设定了一个最小置信度阈值，该阈值用于确定一个关联规则是否被认为是有趣的。
影响： min_conf 较高的值将导致只有高置信度的规则被接受，过滤掉较弱的规则。较低的值将允许具有较低置信度的规则被考虑。
选择这些参数的适当值取决于数据集的具体特征和分析目标：

较高的 min_supp:

优点： 导致较少但更重要的规则。减少要考虑的规则数量。
缺点： 可能错过一些具有较低支持度的潜在有趣规则。
较低的 min_supp:

优点： 捕获更多具有较低支持度的规则。
缺点： 可能导致更多的规则需要分析，包括较不显著的规则。
较高的 min_conf:

优点： 确保只有高置信度的规则被考虑，减少假阳性的风险。
缺点： 可能会过滤掉一些具有较低置信度的潜在有趣规则。
较低的 min_conf:

优点： 捕获更多具有较低置信度的规则。
缺点： 增加包括不太可靠的规则的风险，这些规则可能在实际中不太有用。

• mlxtend というライブラリに apriori が実装されている
• https://rasbt.github.io/mlxtend/
• アソシエーション分析については User Guide や API の
frequent_patterns を参照のこと

In [21]:
from mlxtend.frequent_patterns import apriori, fpgrowth
# 相関ルールを求める関数
from mlxtend.frequent_patterns import association_rules
# apriori の代わりに fpgrowth でもよい
groceries_itemset = apriori(groceries, min_support=0.01,use_colnames=True)
# metric や閾値を設定してルールを作成
association_rules(groceries_itemset, metric="confidence",
min_threshold=0.1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(frankfurter),(sausage),0.058973,0.093950,0.010066,0.170690,1.816810,0.004526,1.092534,0.477760
1,(sausage),(frankfurter),0.093950,0.058973,0.010066,0.107143,1.816810,0.004526,1.053950,0.496203
2,(frankfurter),(root vegetables),0.058973,0.108998,0.010168,0.172414,1.581800,0.003740,1.076627,0.390859
3,(frankfurter),(other vegetables),0.058973,0.193493,0.016472,0.279310,1.443519,0.005061,1.119077,0.326504
4,(frankfurter),(whole milk),0.058973,0.255516,0.020539,0.348276,1.363029,0.005470,1.142330,0.283031
...,...,...,...,...,...,...,...,...,...,...
455,"(rolls/buns, whole milk)",(yogurt),0.056634,0.139502,0.015557,0.274686,1.969049,0.007656,1.186380,0.521686
456,(yogurt),"(rolls/buns, whole milk)",0.139502,0.056634,0.015557,0.111516,1.969049,0.007656,1.061770,0.571925
457,"(soda, yogurt)",(whole milk),0.027351,0.255516,0.010473,0.382900,1.498535,0.003484,1.206423,0.342037
458,"(yogurt, whole milk)",(soda),0.056024,0.174377,0.010473,0.186933,1.072003,0.000703,1.015442,0.071153
